In [1]:
from transformers import pipeline

# Text generation model
generator = pipeline("text-generation", model="distilgpt2")

# Sentiment analysis model
sentiment_analyzer = pipeline("sentiment-analysis")


C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|████████████████████| 76/76 [00:00<00:00, 237.56it/s, Materializing param=transformer.wte.weight]
GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f.
Using a pipeline without specifying a model name and revision in production is not recommended.
Loading weights: 100%|███████████████████| 104/10

In [2]:
def detect_mood(description):
    result = sentiment_analyzer(description)[0]
    if result['label'] == "NEGATIVE":
        return "moody"
    return "happy"


In [14]:
def get_keywords(description):
    prompt = f"Describe the vibe of this scene in a few words: {description}."

    output = generator(
        prompt,
        max_new_tokens=20,   # use this instead of max_length
        do_sample=True,
        temperature=0.8,
        pad_token_id=50256
    )[0]['generated_text']

    words = output.split(":")[-1].replace("\n", " ").split()
    banned = ["women", "girl", "man", "boy", "person", "people"]
    clean_words = [
    w.strip(",.").lower() for w in words
    if w.isalpha() and 4 <= len(w) <= 10 and w.lower() not in banned
]


    return clean_words[:3]


In [31]:
import random

def generate_captions(description):
    mood_type = detect_mood(description)
    keywords = get_keywords(description)

    if len(keywords) < 3:
        keywords = ["dreamy", "calm", "aesthetic"]

    extras = ["moments", "vibes", "energy", "mood", "aura", "magic"]
    emojis_happy = ["✨", "🌸", "🌞", "💫", "🌈"]
    emojis_moody = ["🌙", "🖤", "🌫️", "🌌", "🍂"]

    if mood_type == "happy":
        captions = [
            f"Caught in a {keywords[0]} kind of beautiful moment {random.choice(emojis_happy)}",
            f"Smiling through these {keywords[1]} little things {random.choice(emojis_happy)}",
            f"Just me, some {keywords[2]} vibes, and good energy {random.choice(emojis_happy)}"
        ]
    else:
        captions = [
            f"Lost in {keywords[0]} thoughts under quiet skies {random.choice(emojis_moody)}",
            f"Some nights just feel {keywords[1]} and still {random.choice(emojis_moody)}",
            f"Wrapped in shadows, silence, and {keywords[2]} moments {random.choice(emojis_moody)}"
        ]

    # Extra aesthetic caption line
    extra_line = f"Just {keywords[0]} {random.choice(extras)} and peaceful moments 🌿"
    captions.append(extra_line)

    return captions[:3]


In [23]:
generate_captions("girl standing alone at night in fog on an empty road")


Both `max_new_tokens` (=20) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


['Lost in dreamy thoughts under quiet skies 🌙',
 'Some nights just feel calm and still 🍂',
 'Wrapped in shadows, silence, and aesthetic moments 🌌']

In [24]:
generate_captions("girl with curly hair smiling in a cafe eating a choclate ")

Both `max_new_tokens` (=20) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


['Caught in a with kind of moment 🌞',
 'Smiling through these curly little things ✨',
 'Just me, some hair vibes, and good energy 🌈']

In [39]:
description = "a girl sitting in the hall and eating a pizza "

mood = detect_mood(description)
captions = generate_captions(description)

print("Detected Mood:", mood.upper())
print("\nGenerated Instagram Captions:\n")

for i, caption in enumerate(captions, 1):
    print(f"Caption {i}: {caption}")
    print("-"*50)


Both `max_new_tokens` (=20) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Detected Mood: HAPPY

Generated Instagram Captions:

Caption 1: Caught in a sitting kind of moment 🌸
--------------------------------------------------
Caption 2: Smiling through these hall little things ✨
--------------------------------------------------
Caption 3: Just me, some eating vibes, and good energy ✨
--------------------------------------------------


In [36]:
def caption_style_summary(captions):
    print("\nCaption Style Summary:")
    for c in captions:
        if "🌙" in c or "🖤" in c:
            print(f"'{c}' → Moody Style")
        else:
            print(f"'{c}' → Happy/Aesthetic Style")


In [37]:
caption_style_summary(captions)



Caption Style Summary:
'Caught in a sitting kind of moment ✨' → Happy/Aesthetic Style
'Smiling through these hall little things 🌞' → Happy/Aesthetic Style
'Just me, some eating vibes, and good energy 🌸' → Happy/Aesthetic Style
